# Photo organizer

This script searches all directories, starting with a parent (e.g. locally mounted DropBox), and copies all photos to a master directory

In [14]:
import os
import shutil
import hashlib
import json
import time
from pathlib import Path
from PIL import Image


In [15]:
# Configuration

# This is for a Mac

user = 'bcc'

DIR_DROPBOX = Path('/Users') / user / 'Dropbox'

DIR_SOURCE = DIR_DROPBOX / 'Photos-Extra'
DIR_TARGET = DIR_DROPBOX / 'THE-BIG-PHOTO-DIRECTORY'

DIR_LOG = Path('..') / 'data'
LOG_FILE = DIR_LOG / "search_log.json"  
MIN_WIDTH, MIN_HEIGHT = 640, 480  # Minimum image size

# Ensure target directory exists
os.makedirs(DIR_TARGET, exist_ok=True)

In [16]:
# Functions

def get_short_hash(text):
    """Generate a short hash from text (used for file deduplication)."""
    return hashlib.md5(text.encode()).hexdigest()[:8]  # 8-character hash

def get_timestamp():
    """Return the current timestamp in YYYY-MM-DD HH:MM:SS format."""
    return time.strftime("%Y-%m-%d %H:%M:%S")

def is_valid_image(file_path):
    """Check if the image meets the size criteria."""
    try:
        with Image.open(file_path) as img:
            width, height = img.size
            return width >= MIN_WIDTH and height >= MIN_HEIGHT
    except Exception:
        return False

def load_log():
    """Load previously searched directories and moved file hashes."""
    if LOG_FILE.exists():
        with open(LOG_FILE, "r") as f:
            return json.load(f)
    return {"searched_dirs": [], "moved_files": {}}

def save_log(log_data):
    """Save searched directories and moved file records."""
    with open(LOG_FILE, "w") as f:
        json.dump(log_data, f, indent=4)


In [17]:
# The main function

def process_directory(directory: Path, log_data):
    """Search for images in a directory and move valid ones using Pathlib."""
    searched_dirs = set(log_data["searched_dirs"])
    moved_files = log_data["moved_files"]  # Dictionary: {original_path: {"hash": hash, "timestamp": timestamp}}

    for file_path in directory.rglob("*"):  # Recursively find all files
        if file_path.is_file() and file_path.suffix.lower() in {".jpg", ".jpeg", ".heic", ".png"}:
            
            # Skip if already moved
            if str(file_path) in moved_files:
                continue

            # Process only new images
            if is_valid_image(file_path):
                file_hash = get_short_hash(str(file_path))
                new_name = f"{file_path.stem}_{file_hash}{file_path.suffix}"
                target_path = Path(DIR_TARGET) / new_name

                shutil.move(file_path, target_path)  # Move instead of copy
                print(f"Moved: {file_path} -> {target_path}")

                # Store moved file details
                moved_files[str(file_path)] = {
                    "hash": file_hash,
                    "new_path": str(target_path),
                    "timestamp": get_timestamp(),
                }

            # Track parent directory as searched
            searched_dirs.add(str(file_path.parent))

    log_data["searched_dirs"] = list(searched_dirs)
    log_data["moved_files"] = moved_files

    save_log(log_data)  # Save log after processing


In [18]:

log_data = load_log()

process_directory(DIR_SOURCE, log_data)

print("Done.")


Moved: /Users/bcc/Dropbox/Photos-Extra/CCHS.photoslibrary/resources/proxies/derivatives/03/00/33a/UNADJUSTEDNONRAW_thumb_33a.jpg -> /Users/bcc/Dropbox/THE-BIG-PHOTO-DIRECTORY/UNADJUSTEDNONRAW_thumb_33a_c0272e6d.jpg
Moved: /Users/bcc/Dropbox/Photos-Extra/CCHS.photoslibrary/resources/proxies/derivatives/03/00/354/UNADJUSTEDNONRAW_thumb_354.jpg -> /Users/bcc/Dropbox/THE-BIG-PHOTO-DIRECTORY/UNADJUSTEDNONRAW_thumb_354_63f10e47.jpg
Moved: /Users/bcc/Dropbox/Photos-Extra/CCHS.photoslibrary/resources/proxies/derivatives/03/00/398/UNADJUSTEDNONRAW_thumb_398.jpg -> /Users/bcc/Dropbox/THE-BIG-PHOTO-DIRECTORY/UNADJUSTEDNONRAW_thumb_398_8cecece6.jpg
Moved: /Users/bcc/Dropbox/Photos-Extra/CCHS.photoslibrary/resources/proxies/derivatives/03/00/353/UNADJUSTEDNONRAW_thumb_353.jpg -> /Users/bcc/Dropbox/THE-BIG-PHOTO-DIRECTORY/UNADJUSTEDNONRAW_thumb_353_43a79339.jpg
Moved: /Users/bcc/Dropbox/Photos-Extra/CCHS.photoslibrary/resources/proxies/derivatives/03/00/33f/UNADJUSTEDNONRAW_thumb_33f.jpg -> /Users/b